<a href="https://colab.research.google.com/github/Naina-j04/Garbage_classifier/blob/main/GarbageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torchvision
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
#pointing to dataset
data_dir  = '/content/drive/MyDrive/Garbage_classification/Data'

classes = os.listdir(data_dir)
print(classes)

['plastic', 'trash', 'paper', 'glass', 'metal', 'cardboard', 'garbage.pth', 'Models', '.ipynb_checkpoints']


In [4]:
#resizing and normalizing the dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = ImageFolder(data_dir, transform = transformations)

In [5]:
len(dataset)


2527

In [6]:
train_data,val_data,test_data=random_split(dataset,[1616,405,506])
len(train_data),len(val_data),len(test_data)

(1616, 405, 506)

In [12]:
class GarbageCNN(nn.Module):
  def __init__(self,classes=6):
    super (GarbageCNN,self).__init__()

    self.c1=nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,padding=1)
    self.c2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=1)
    self.c3=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)
    self.c4=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1)
    self.c5=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1)

    self.pool=nn.MaxPool2d(kernel_size=2,stride=2)

    self.fc = nn.Linear(256 * 7 * 7, classes)

  def forward(self,x):
    x=self.pool(F.relu(self.c1(x)))
    x=self.pool(F.relu(self.c2(x)))
    x=self.pool(F.relu(self.c3(x)))
    x=self.pool(F.relu(self.c4(x)))
    x=self.pool(F.relu(self.c5(x)))
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x
garbage=GarbageCNN()


In [13]:
from torch.utils.data import DataLoader
train_loader=DataLoader(train_data,batch_size=32,shuffle=True)
val_loader=DataLoader(val_data,batch_size=32,shuffle=False)
test_loader=DataLoader(test_data,batch_size=32,shuffle=False)

In [14]:
import torch.optim as optim
#starting the training
classes=6
model=GarbageCNN(classes)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001,)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cuda')

In [21]:
num_epochs = 20
print("Starting Training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')

print('Finished Training.')


Starting Training...
Epoch [1/20], Training Loss: 1.6028, Validation Loss: 1.4459, Validation Accuracy: 39.26%
Epoch [2/20], Training Loss: 1.3236, Validation Loss: 1.2293, Validation Accuracy: 52.59%
Epoch [3/20], Training Loss: 1.1350, Validation Loss: 1.1784, Validation Accuracy: 56.30%
Epoch [4/20], Training Loss: 1.0486, Validation Loss: 0.9972, Validation Accuracy: 61.73%
Epoch [5/20], Training Loss: 0.9315, Validation Loss: 0.9922, Validation Accuracy: 64.69%
Epoch [6/20], Training Loss: 0.8350, Validation Loss: 1.0390, Validation Accuracy: 61.23%
Epoch [7/20], Training Loss: 0.7677, Validation Loss: 0.9685, Validation Accuracy: 64.69%
Epoch [8/20], Training Loss: 0.7103, Validation Loss: 0.9857, Validation Accuracy: 65.93%
Epoch [9/20], Training Loss: 0.5775, Validation Loss: 0.9572, Validation Accuracy: 69.63%
Epoch [10/20], Training Loss: 0.5144, Validation Loss: 0.9002, Validation Accuracy: 68.89%
Epoch [11/20], Training Loss: 0.4177, Validation Loss: 1.0215, Validation Accu

In [22]:
# FINAL TEST EVALUATION
print('Starting Final Evaluation on Test Set...')
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        test_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_test_loss = test_loss / len(test_loader)
accuracy = 100 * correct / total
print(f'Final Test Loss: {avg_test_loss:.4f}, Final Test Accuracy: {accuracy:.2f}%')

Starting Final Evaluation on Test Set...
Final Test Loss: 1.4883, Final Test Accuracy: 72.33%


In [34]:
path='/content/drive/MyDrive/Garbage_classification/Data/bettermodel.pth'
torch.save(model.state_dict(), path)